In [16]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 1
%aimport utils, GroupingAlgorithm

import numpy as np
import networkx as nx
import warnings
from itertools import permutations
from utils import number2SummedOp, get_backend_connectivity, Label2Chain, save_object, molecules
from GroupingAlgorithm import groupingWithOrder, TPBgrouping
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit import IBMQ

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') 

ibmqfactory.load_account:WARNING:1970-01-01 13:10:30,770: Credentials are already in use. The existing account in the session will be replaced.


# Optimal grouping

In [22]:
backend_device = provider.get_backend(name_backend)

WC_device = get_backend_connectivity(backend_device)

molecules_names = ['H2', 'LiH', 'BeH2', 'H2O']

for molecule in molecules_names:
    name_backend = 'ibmq_montreal'
    file_name = 'optimal_grouping_' + molecule + '_' + name_backend    
    
    data_qubit_op = np.load('../data/' + file_name + '.npy', allow_pickle=True).item()
    qubit_op = number2SummedOp(data_qubit_op['optimal_labels'], data_qubit_op['optimal_coeffs'])
    num_qubits = qubit_op.num_qubits
    paulis, _, _ = Label2Chain(qubit_op)
    
    WC_ideal = list(permutations(list(range(num_qubits)), 2))
    
    n_groups = {'TPB': len(TPBgrouping(paulis)[1]),
                'EM': len(groupingWithOrder(paulis, WC_ideal)[0]),
                'HEEM': len(groupingWithOrder(paulis, WC_device)[0])}
    data_qubit_op['n_groups'] = n_groups
    
    _, init_state = molecules(molecule, initial_state=True)
    data_qubit_op['init_state'] = init_state
    
    save_object(data_qubit_op, file_name)

Do you want to overwrite the file (../data/optimal_grouping_H2_ibmq_montreal.npy)?  [y]/n:  


data saved


Do you want to overwrite the file (../data/optimal_grouping_LiH_ibmq_montreal.npy)?  [y]/n:  


data saved


Do you want to overwrite the file (../data/optimal_grouping_BeH2_ibmq_montreal.npy)?  [y]/n:  


data saved


Do you want to overwrite the file (../data/optimal_grouping_H2O_ibmq_montreal.npy)?  [y]/n:  


data saved
